In [5]:
import pandas as pd
import numpy as np

import os

In [6]:
INTERNAL_DIR = 'internal_data'
EXTERNAL_DIR = 'external_data'

INTERNAL_FILENAME = 'internal_df.csv'
EXTERNAL_FILENAME = 'external_df.csv'

In [7]:
INTERNAL_PATH = os.path.join('..', INTERNAL_DIR, INTERNAL_FILENAME)
EXTERNAL_PATH = os.path.join('..', EXTERNAL_DIR, EXTERNAL_FILENAME)

In [26]:
internal_df = pd.read_csv(INTERNAL_PATH, index_col=False)
external_df = pd.read_csv(EXTERNAL_PATH, index_col=False)

In [27]:
covid_confirmation_df = internal_df[['사용일자', '확진자수']].drop_duplicates()
covid_confirmation_df.head()

,사용일자,확진자수
0,2020-01-20,1
40,2020-01-21,1
88,2020-01-22,1
158,2020-01-23,1
227,2020-01-24,2


In [28]:
internal_df = internal_df.iloc[:, :-1]

In [23]:
internal_df.head()

,사용일자,대분류,중분류,중분류_카운트
0,2020-01-20,관광,관광여행,23
1,2020-01-20,유흥주점,유흥주점,3494
2,2020-01-20,유흥주점,칵테일바,587
3,2020-01-20,음식점,스넥,72529
4,2020-01-20,음식점,일반한식,274845


In [4]:
external_df.head()

,사용일자,중분류_카운트,중분류
0,2020-01-01,2509091,버스
1,2020-01-02,5286628,버스
2,2020-01-03,5580078,버스
3,2020-01-04,4036695,버스
4,2020-01-05,3138649,버스


In [33]:
external_df['대분류'] = np.nan

In [46]:
external_df.loc[external_df['중분류'] == '교통량', ['대분류']] = '교통'
external_df.loc[external_df['중분류'] != '교통량', ['대분류']] = '대중교통'

In [47]:
external_df

,사용일자,중분류_카운트,중분류,대분류
0,2020-01-01,2509091,버스,대중교통
1,2020-01-02,5286628,버스,대중교통
2,2020-01-03,5580078,버스,대중교통
3,2020-01-04,4036695,버스,대중교통
4,2020-01-05,3138649,버스,대중교통
...,...,...,...,...
541,2020-06-26,9568914,교통량,교통
542,2020-06-27,8964257,교통량,교통
543,2020-06-28,7635665,교통량,교통
544,2020-06-29,9052687,교통량,교통


In [50]:
covid_confirmation_df

,사용일자,확진자수
0,2020-01-20,1
40,2020-01-21,1
88,2020-01-22,1
158,2020-01-23,1
227,2020-01-24,2
...,...,...
9056,2020-06-10,11902
9124,2020-06-11,11947
9192,2020-06-12,12003
9261,2020-06-13,12051


In [57]:
concated_df = pd.concat([internal_df, external_df], axis = 0)
concated_df

,사용일자,대분류,중분류,중분류_카운트
0,2020-01-20,관광,관광여행,23
1,2020-01-20,유흥주점,유흥주점,3494
2,2020-01-20,유흥주점,칵테일바,587
3,2020-01-20,음식점,스넥,72529
4,2020-01-20,음식점,일반한식,274845
...,...,...,...,...
541,2020-06-26,교통,교통량,9568914
542,2020-06-27,교통,교통량,8964257
543,2020-06-28,교통,교통량,7635665
544,2020-06-29,교통,교통량,9052687


In [69]:
median_v_series = concated_df.groupby(['대분류', '중분류'], as_index = False)['중분류_카운트'].median()
median_v_series.rename(columns = {'중분류_카운트' : '중분류_대표값'}, inplace = True)

In [75]:
combined_median_df = pd.merge(concated_df, median_v_series, on = ['대분류', '중분류'])
combined_median_df.head(2)

,사용일자,대분류,중분류,중분류_카운트,중분류_대표값
0,2020-01-20,관광,관광여행,23,216.0
1,2020-01-21,관광,관광여행,1036,216.0


In [76]:
combined_median_df['증가추이'] = (combined_median_df['중분류_카운트'] / combined_median_df['중분류_대표값']) * 100

In [97]:
combined_median_df.loc[(combined_median_df['중분류_대표값'] == 0) & (combined_median_df['중분류_카운트'] == 0), ['증가추이']] = 0

In [ ]:
combined_median_df.to_csv('result_df.csv', index = False)